In [163]:
from deap import creator, base, tools, algorithms
import random
import numpy

In [164]:
with open("11.txt") as f:
    line1 = f.readline().rstrip() # сразу удаляем перенос строки в конце
    MAX_WEIGHT = int(line1.split(' ')[0]) # грузоподъемность
    MAX_VOLUME = int(line1.split(' ')[1]) # вместимость
    items = {}
    for num, line in enumerate(f):
        items[num] = tuple(map(float, line.rstrip().split(' ')))
    
IND_INIT_SIZE = 5 # начальное количество предметов особи
MAX_ITEM = 50   # максимальное количество предметов в рюкзаке (в особи)
NBR_ITEMS = 20 # количество предметов

In [165]:
# Инициализируем популяцию и существ

# функция приспособленности: минимизация параметра 1 при максимизации параметра 2
creator.create("Fitness", base.Fitness, weights=(-1.0, -1.0, 1.0))
# особь: неупорядоченный набор признаков
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()
# конструирование признака особи: случайное число от 1 до 19
toolbox.register("attr_item", random.randrange, NBR_ITEMS)
# конструирование особи: помещаем в особь атрибуты числом 5
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
# конструирование популяции: список особей
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [166]:
# Подсчет суммарного веса и стоимости
def evalKnapsack(individual):
    weight = 0.0
    volume = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        volume += items[item][1]
        value += items[item][2]        
    if (len(individual) > MAX_ITEM) or (weight > MAX_WEIGHT) or (volume > MAX_VOLUME):
        return 10000, 10000, 0,             # Ensure overweighted bags are dominated
    return weight, volume, value

# Оператор наследования, определяет детей двух множеств: первый как пересечение, второй как разность
def cxSet(ind1, ind2):
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

# Оператор мутации, рандомно добавляет и удаляет элементы
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

# Регистрируем операции
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [167]:
def main():
    random.seed(64)
    NGEN = 50   # Количество поколений
    MU = 50    # Количество особей, переходящих в следующее поколение
    LAMBDA = 100 # Количество потомков, появляющихся к следующему поколению
    CXPB = 0.7  # Вероятность появления потомка кроссовером
    MUTPB = 0.3 # Вероятность появления потомка мутацией
    
    # популяция: список особей (под капотом каждая особь - множество признаков-чисел от 0 до 20)
    pop = toolbox.population(n=MU)
    # передний зал славы, содержит особей, наилучших из тех, которые никогда не доминировали. отсортирован по ухудшению (первый - самый лучший и т.д.)
    hof = tools.ParetoFront()
    # статистика списка всех особей поколения, узнаем среднее, стандартное отклонение, минимум, максимум
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    # Эволюционная стратегия "Мю плюс лямбда"
    pop100 = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, 60, stats, halloffame=hof, verbose=1)

main()

gen	nevals	avg                                  	max                            
0  	50    	[4.38842e+03 3.60000e+00 1.14628e+03]	[7.148e+03 4.800e+00 1.640e+03]
1  	100   	[2.8503e+03 2.2960e+00 8.5798e+02]   	[6.935e+03 5.200e+00 1.844e+03]
2  	100   	[3.1298e+03 2.4860e+00 9.7400e+02]   	[6.935e+03 5.200e+00 1.955e+03]
3  	100   	[3.22088e+03 2.60000e+00 1.01716e+03]	[6.935e+03 5.200e+00 1.955e+03]
4  	100   	[3.36364e+03 2.67000e+00 1.05378e+03]	[6.935e+03 5.200e+00 1.955e+03]
5  	100   	[3.55476e+03 2.75000e+00 1.09380e+03]	[8.027e+03 5.800e+00 2.121e+03]
6  	100   	[3.54162e+03 2.85400e+00 1.12056e+03]	[8.027e+03 5.800e+00 2.121e+03]
7  	100   	[3.51598e+03 2.77600e+00 1.10488e+03]	[8.027e+03 5.600e+00 2.127e+03]
8  	100   	[3.65926e+03 2.86000e+00 1.14288e+03]	[9.598e+03 6.500e+00 2.471e+03]
9  	100   	[4.03300e+03 3.19600e+00 1.25502e+03]	[9.598e+03 6.500e+00 2.471e+03]
10 	100   	[4.10624e+03 3.16400e+00 1.26812e+03]	[1.0457e+04 7.1000e+00 2.6440e+03]
11 	100   	[4.02192e+03 3